In [2]:
! pip3 install edgetpu

ERROR: Could not find a version that satisfies the requirement edgetpu (from versions: none)
ERROR: No matching distribution found for edgetpu

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [1]:
#!/usr/bin/env python
#
# Test Posenet model with custom decoding
# ----------------------------------------------------------------------------------------------------------------------
import cv2
import numpy as np
from edgetpu.basic.basic_engine import BasicEngine
from posenet.decode_multi import decode_multiple_poses

MODEL = "/tmp/output_model.tflite"
OUTPUT_STRIDE = 16
NUM_KEYPOINTS = 17


def extract_outputs(outputs, engine):
    """Extract heatmaps, offsets, and displacement vectors"""

    # Input image and heatmap dimensions
    _, img_h, img_w, channels = engine.get_input_tensor_shape()
    height = int(1 + (img_w - 1) / OUTPUT_STRIDE)
    width = int(1 + (img_h - 1) / OUTPUT_STRIDE)

    # Reshape output tensors
    out_sz = engine.get_all_output_tensors_sizes()

    # Heatmaps
    ofs = 0
    heatmaps = outputs[ofs: ofs + out_sz[0]].reshape(height, width, NUM_KEYPOINTS)
    ofs += out_sz[0]

    # Offsets - [height, width, 2, 17]
    offsets = outputs[ofs: ofs + out_sz[1]].reshape(height, width, NUM_KEYPOINTS * 2)
    ofs += out_sz[1]

    # Displacement vectors (FWD, BWD): size [height, width, 4, 16], columns [fwd_i, fwd_j, bwd_i, bwd_j]
    raw_dsp = outputs[ofs: ofs + out_sz[2]].reshape(height, width, 4, -1)
    fwd = raw_dsp[:, :, 0:2, :]
    bwd = raw_dsp[:, :, 2:4, :]

    return {
        'heatmaps': 1. / (1. + np.exp(-heatmaps)),  # apply sigmoid function to heatmaps
        'offsets': offsets,
        'displacements_fwd': fwd,
        'displacements_bwd': bwd
    }


# ----------------------------------------------------------------------------------------------------------------------
# Testing
# ----------------------------------------------------------------------------------------------------------------------
INPUT_IMG = '/home/gianni/stura/embed/coral/python/project-posenet/couple.jpg'

def main():
    # Read input image
    frame = cv2.imread(INPUT_IMG)
    frame = cv2.resize(frame, (481, 353))
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Instantiate inference engine
    engine = BasicEngine(MODEL)
    inference_time, output = engine.run_inference(frame.flatten())
    print('Done, inference time: %d msec, output shape: %s' % (inference_time, output.shape))

    # Decode pose
    out = extract_outputs(output, engine)
    pose_scores, keypoint_scores, keypoint_coords = decode_multiple_poses(
        out['heatmaps'],
        out['offsets'],
        out['displacements_fwd'],
        out['displacements_bwd'],
        OUTPUT_STRIDE,
        max_pose_detections=10, score_threshold=0.5, nms_radius=20, min_pose_score=0.1
    )
    print('pose_scores: %s' % str(pose_scores))


if __name__ == '__main__':
    main()

ModuleNotFoundError: No module named 'edgetpu'